# Image **Classification**

Classifying images

Author: Shreyash Gupta

Organization: IndiaMART InterMESH Pvt. Ltd.

# **Referring to data**

Importing necessary modules

In [0]:
import os
from google.colab import drive

Mounting to Google Drive

In [0]:
drive.mount('/content/drive')

Changing working directory

In [0]:
os.chdir('/content/drive/My Drive/Colab Notebooks')

Defining training and validation directory paths

In [0]:
train__2419_dir = os.path.join("/automobile/train/2419")
train_2579_dir = os.path.join("/automobile/train/2579")
train_2956_dir = os.path.join("/automobile/train/2956")
train_3029_dir = os.path.join("/automobile/train/3029")
val__2419_dir = os.path.join("/automobile/val/2419")
val_2579_dir = os.path.join("/automobile/val/2579")
val_2956_dir = os.path.join("/automobile/val/2956")
val_3029_dir = os.path.join("/automobile/val/3029")

# **Building the model**

Importing necessary modules

In [0]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

Getting pretrained weights for InceptionV3 model

In [0]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

Loading the InceptionV3 model

In [0]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
pre_trained_model = InceptionV3(input_shape = (300, 300, 3),
                               include_top = False,
                               weights = None)

local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
  layer.trainable = False
  
pre_trained_model.summary()

Fetching the last layer and output

In [0]:
last_layer = pre_trained_model.get_layer("mixed7")
last_output = last_layer.output

Building the core model

In [0]:
x = tf.keras.layers.Flatten()(last_output)
x = tf.keras.layers.Dense(1024, activation = "relu")(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(4, activation = "softmax")(x)
model = tf.keras.Model(pre_trained_model.input, x)

Compiling the model

In [0]:
model.compile(loss = "categorical_crossentropy",
              optimizer = Adam(lr = 1e-4),
              metrics = ["acc"])

# **Generating training and validation data**

Importing necessary modules

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Defining training data generator

In [0]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 90,
    width_shift_range = 0.3,
    height_shift_range = 0.3,
    shear_range = 0.3,
    zoom_range = 0.3,
    horizontal_flip = True,
    fill_mode = "nearest")

Defininig validation data generator

In [0]:
validation_datagen = ImageDataGenerator(rescale = 1./255)

Getting training data

In [0]:
train_generator = train_datagen.flow_from_directory(
    "/content/drive/My Drive/Colab Notebooks/automobile/train/",
    target_size = (300, 300),
    batch_size = 128,
    class_mode = "categorical")

Getting validation data

In [0]:
validation_generator = validation_datagen.flow_from_directory(
    "/content/drive/My Drive/Colab Notebooks/automobile/val/",
    target_size = (300, 300),
    batch_size = 32,
    class_mode = "categorical")

# **Training and visualizing the model**

Importing necessary modules

In [0]:
import matplotlib.pyplot as plt

Fitting the model

In [0]:
history = model.fit_generator(
    train_generator,
    epochs = 100,
    verbose = 1,
    validation_data = validation_generator)

Defining variables for plotting

In [0]:
accuracy = history.history["acc"]
validation_accuracy = history.history["val_acc"]
loss = history.history["loss"]
validation_loss = history.history["val_loss"]
epochs = range(len(accuracy))

Plotting accuracy

In [0]:
plt.plot(epochs, accuracy, "r", label = "Training Accuracy")
plt.plot(epochs, validation_accuracy, "b", label = "Validation Accuracy")
plt.title("Accuracy")
plt.legend()

In [0]:
plt.figure()

Plotting loss

In [0]:
plt.plot(epochs, loss, "r", label = "Training Loss")
plt.plot(epochs, validation_loss, "b", label = "Validation Loss")
plt.title("Loss")
plt.legend()

In [0]:
plt.show()

# Prediction

Defining a prediction function

In [0]:
def predict(model, img):
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    preds = model.predict(x)
    return preds[0]
  


Predicting on an image

In [0]:
img = image.load_img('/automobile/train/2419/135398099.jpg', target_size=(300, 300))
preds = predict(model, img)
preds